# Topic Similarity: with LDA model & Cossine similarity

## Import Packages

In [1]:
import numpy as np
import pandas as pd
import spacy
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.matutils import cossim
from gensim.models import CoherenceModel

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from pprint import pprint

import os
import re

import pyLDAvis.gensim
import pickle 
import pyLDAvis


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Read Data

In [24]:
df_First = pd.read_excel('Data_files/New_1st_debate.xlsx', engine="openpyxl")
df_Second = pd.read_excel('Data_files/New_2nd_debate.xlsx', engine="openpyxl")

df_All = df_First.append(df_Second)
print(df_First.shape)
print(df_Second.shape)
print(df_All.shape)

(789, 5)
(514, 3)
(1303, 5)


## Data Processing

In [3]:
# standard stopwords
my_stopwords = stopwords.words('english')
# additional stopwords
my_stopwords.extend(['from', 'subject', 're', 'edu', 'use'])

# Remove punctuation
df_First['text_processed'] = df_First['text'].map(lambda x: re.sub('[,\.!?]', '', x))
df_Second['text_processed'] = df_Second['text'].map(lambda x: re.sub('[,\.!?]', '', x))
df_All['text_processed'] = df_All['text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert lowercase
df_First['text_processed'] = df_First['text_processed'].map(lambda x: x.lower())
df_Second['text_processed'] = df_Second['text_processed'].map(lambda x: x.lower())
df_All['text_processed'] = df_All['text_processed'].map(lambda x: x.lower())

<>:7: DeprecationWarning: invalid escape sequence \.
<>:8: DeprecationWarning: invalid escape sequence \.
<>:9: DeprecationWarning: invalid escape sequence \.
<>:7: DeprecationWarning: invalid escape sequence \.
<>:8: DeprecationWarning: invalid escape sequence \.
<>:9: DeprecationWarning: invalid escape sequence \.
<ipython-input-3-8a0f87e8d8ac>:7: DeprecationWarning: invalid escape sequence \.
  df_First['text_processed'] = df_First['text'].map(lambda x: re.sub('[,\.!?]', '', x))
<ipython-input-3-8a0f87e8d8ac>:8: DeprecationWarning: invalid escape sequence \.
  df_Second['text_processed'] = df_Second['text'].map(lambda x: re.sub('[,\.!?]', '', x))
<ipython-input-3-8a0f87e8d8ac>:9: DeprecationWarning: invalid escape sequence \.
  df_All['text_processed'] = df_All['text'].map(lambda x: re.sub('[,\.!?]', '', x))


In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


dataF = df_First.text_processed.values.tolist()
data_words_F = list(sent_to_words(dataF))
data_words_F = [[x for x in word if len(x) > 4] for word in data_words_F]
data_words_F

[['evening',
  'health',
  'education',
  'campus',
  'western',
  'reserve',
  'university',
  'cleveland',
  'clinic',
  'chris',
  'wallace',
  'welcome',
  'first',
  'presidential',
  'debates',
  'between',
  'president',
  'donald',
  'trump',
  'former',
  'president',
  'biden',
  'debate',
  'sponsored',
  'commission',
  'presidential',
  'debates',
  'commission',
  'designed',
  'format',
  'roughly',
  'minute',
  'segments',
  'minute',
  'answers',
  'candidate',
  'first',
  'question',
  'discussion',
  'segment',
  'campaigns',
  'agreed',
  'these',
  'rules',
  'record',
  'decided',
  'topics',
  'questions',
  'topic',
  'assure',
  'questions',
  'shared',
  'commission',
  'candidates'],
 ['debate',
  'being',
  'conducted',
  'under',
  'health',
  'safety',
  'protocols',
  'designed',
  'cleveland',
  'clinic',
  'which',
  'serving',
  'health',
  'security',
  'advisor',
  'commission',
  'debates',
  'precaution',
  'campaigns',
  'agreed',
  'candidates'

In [5]:
dataS = df_Second.text_processed.values.tolist()
data_words_S = list(sent_to_words(dataS))
data_words_S = [[x for x in word if len(x) > 4] for word in data_words_S]
data_words_S

[['evening',
  'everyone',
  'evening',
  'thank',
  'being',
  'honor',
  'moderate',
  'debate',
  'tonight',
  'final',
  'debate',
  'welcome',
  'first',
  'family',
  'first',
  'thankful',
  'feeling',
  'better',
  'welcome',
  'biden',
  'family',
  'biden',
  'thank',
  'being',
  'tonight',
  'excited',
  'looking',
  'forward',
  'really',
  'robust',
  'discussion',
  'thing',
  'would',
  'reiterate',
  'guidelines',
  'candidates',
  'talking',
  'please',
  'applause',
  'other',
  'reactions',
  'except',
  'course',
  'cheer',
  'applause',
  'everyone',
  'thank',
  'having',
  'really',
  'honor',
  'lifetime',
  'going',
  'organized',
  'settled',
  'start',
  'thank',
  'being',
  'silence',
  'evening',
  'belmont',
  'university',
  'nashville',
  'tennessee',
  'kristen',
  'welker',
  'welcome',
  'final',
  'presidential',
  'debate',
  'between',
  'president',
  'donald',
  'trump',
  'former',
  'president',
  'biden',
  'tonight',
  'debate',
  'sponsore

In [6]:
dataA = df_All.text_processed.values.tolist()
data_words_All = list(sent_to_words(dataA))
data_words_All = [[x for x in word if len(x) > 4] for word in data_words_All]
data_words_All

[['evening',
  'health',
  'education',
  'campus',
  'western',
  'reserve',
  'university',
  'cleveland',
  'clinic',
  'chris',
  'wallace',
  'welcome',
  'first',
  'presidential',
  'debates',
  'between',
  'president',
  'donald',
  'trump',
  'former',
  'president',
  'biden',
  'debate',
  'sponsored',
  'commission',
  'presidential',
  'debates',
  'commission',
  'designed',
  'format',
  'roughly',
  'minute',
  'segments',
  'minute',
  'answers',
  'candidate',
  'first',
  'question',
  'discussion',
  'segment',
  'campaigns',
  'agreed',
  'these',
  'rules',
  'record',
  'decided',
  'topics',
  'questions',
  'topic',
  'assure',
  'questions',
  'shared',
  'commission',
  'candidates'],
 ['debate',
  'being',
  'conducted',
  'under',
  'health',
  'safety',
  'protocols',
  'designed',
  'cleveland',
  'clinic',
  'which',
  'serving',
  'health',
  'security',
  'advisor',
  'commission',
  'debates',
  'precaution',
  'campaigns',
  'agreed',
  'candidates'

## Build the bigram and trigram models

In [7]:
# Build the bigram and trigram models
bigram_F = gensim.models.Phrases(data_words_F, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_F = gensim.models.Phrases(bigram_F[data_words_F], threshold=100)
bigram_S = gensim.models.Phrases(data_words_S, min_count=5, threshold=100) 
trigram_S = gensim.models.Phrases(bigram_S[data_words_S], threshold=100)
bigram_All = gensim.models.Phrases(data_words_All, min_count=5, threshold=100) 
trigram_All = gensim.models.Phrases(bigram_All[data_words_All], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_F = gensim.models.phrases.Phraser(bigram_F)
trigram_mod_F = gensim.models.phrases.Phraser(trigram_F)
bigram_mod_S = gensim.models.phrases.Phraser(bigram_S)
trigram_mod_S = gensim.models.phrases.Phraser(trigram_S)
bigram_mod_All = gensim.models.phrases.Phraser(bigram_All)
trigram_mod_All = gensim.models.phrases.Phraser(trigram_All)

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in my_stopwords] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts, bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops_F = remove_stopwords(data_words_F)
data_words_nostops_S = remove_stopwords(data_words_S)
data_words_nostops_All = remove_stopwords(data_words_All)

# Form Bigrams
data_words_bigrams_F = make_bigrams(data_words_nostops_F, bigram_mod_F)
data_words_bigrams_S = make_bigrams(data_words_nostops_S, bigram_mod_S)
data_words_bigrams_All = make_bigrams(data_words_nostops_All, bigram_mod_All)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

#Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_F = lemmatization(data_words_bigrams_F, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized_S = lemmatization(data_words_bigrams_S, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized_All = lemmatization(data_words_bigrams_All, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


## Create Dictionary & Corpus for LDA model

In [10]:
# Create Dictionary
id2word_F = corpora.Dictionary(data_lemmatized_F)
id2word_S = corpora.Dictionary(data_lemmatized_S)
id2word_All = corpora.Dictionary(data_lemmatized_All)

# Create Corpus
texts_F = data_lemmatized_F
texts_S = data_lemmatized_S
texts_All = data_lemmatized_All

# Term Document Frequency
corpus_F = [id2word_F.doc2bow(text) for text in texts_F]
corpus_S = [id2word_S.doc2bow(text) for text in texts_S]
corpus_All = [id2word_All.doc2bow(text) for text in texts_All]

#print(corpus_F)

## Create LDA model

#### LDA topic model: based on Debate1

In [11]:
# Build First debate LDA model
lda_model_First = gensim.models.LdaMulticore(corpus=corpus_F,
                                       id2word=id2word_F,
                                       num_topics=8, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

#### LDA topic model: based on Debate2

In [12]:
# Build Second debate LDA model
lda_model_Second = gensim.models.LdaMulticore(corpus=corpus_S,
                                       id2word=id2word_S,
                                       num_topics=8, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

#### LDA topic model: based on Both Debates

In [13]:
# Build Combined debate LDA model
lda_model_All = gensim.models.LdaMulticore(corpus=corpus_All,
                                       id2word=id2word_All,
                                       num_topics=8, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

#### Show the top topics from LDA model trained from both debate

In [14]:
# Print the Keyword in the topics
pprint(lda_model_All.print_topics())

[(0,
  '0.031*"industry" + 0.030*"dollar" + 0.030*"go" + 0.023*"excuse" + '
  '0.016*"response" + 0.016*"people" + 0.012*"wrong" + 0.011*"never" + '
  '0.010*"give" + 0.010*"spend"'),
 (1,
  '0.067*"would" + 0.030*"go" + 0.030*"energy" + 0.027*"economy" + '
  '0.025*"create" + 0.023*"happen" + 0.018*"million" + 0.018*"building" + '
  '0.016*"country" + 0.014*"ahead"'),
 (2,
  '0.048*"crosstalk" + 0.033*"people" + 0.025*"go" + 0.023*"number" + '
  '0.022*"year" + 0.019*"state" + 0.017*"minute" + 0.013*"different" + '
  '0.012*"place" + 0.010*"make"'),
 (3,
  '0.076*"people" + 0.040*"country" + 0.018*"number" + 0.018*"talk" + '
  '0.014*"want" + 0.012*"world" + 0.011*"destroy" + 0.011*"bring" + 0.011*"go" '
  '+ 0.010*"together"'),
 (4,
  '0.078*"go" + 0.022*"never" + 0.020*"think" + 0.019*"election" + '
  '0.017*"first" + 0.016*"year" + 0.015*"black" + 0.014*"thing" + '
  '0.014*"matter" + 0.013*"talk"'),
 (5,
  '0.038*"statement" + 0.033*"family" + 0.032*"thing" + 0.027*"section" + '
 

## Visualize topics-keywords of LDA

### Topic: First debate

In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_First, corpus_F, id2word_F)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.142405 -0.113874       1        1  21.108007
2     -0.039556 -0.020726       2        1  16.871881
0     -0.070105  0.108596       3        1  13.912037
5      0.001487  0.060233       4        1  11.689456
3      0.004522 -0.043112       5        1   9.978051
6      0.184692 -0.025037       6        1   9.488564
7      0.004300  0.125634       7        1   9.243153
1      0.057065 -0.091715       8        1   7.708854, topic_info=               Term        Freq       Total Category  logprob  loglift
191          ballot   44.000000   44.000000  Default  30.0000  30.0000
161           ahead   15.000000   15.000000  Default  29.0000  29.0000
276       crosstalk   44.000000   44.000000  Default  28.0000  28.0000
53               go  118.000000  118.000000  Default  27.0000  27.0000
43            right   34.000000   34.000000  Default  26.0000  26.0000
..              ...         ...         ...      ...      ...      ...
250             ask    2.246042   14.479457   Topic8  -4.9384   0.6992
561  administration    1.702698    5.363263   Topic8  -5.2154   1.4154
17         question    2.549406   35.526138   Topic8  -4.8118  -0.0716
53               go    1.949777  118.953094   Topic8  -5.0799  -1.5482
74            place    1.746016   12.743801   Topic8  -5.1903   0.5751

[465 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
212       3  0.375279   absolutely
212       8  0.562918   absolutely
177       2  0.655925       accord
732       3  0.475453  accountable
732       7  0.475453  accountable
...     ...       ...          ...
158       3  0.890420        wrong
151       1  0.361441         year
151       2  0.506018         year
151       3  0.072288         year
151       6  0.072288         year

[734 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 1, 6, 4, 7, 8, 2])

### Topic: Second debate

In [16]:
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_Second, corpus_S, id2word_S)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.166639  0.108261       1        1  22.721504
0      0.047448  0.016027       2        1  16.222324
5      0.094682 -0.014122       3        1  12.868516
2     -0.017221 -0.107017       4        1  12.188457
1     -0.088367 -0.086886       5        1  10.800371
4      0.030732 -0.022004       6        1   9.215701
6      0.083889  0.118927       7        1   8.376447
3      0.015477 -0.013186       8        1   7.606688, topic_info=          Term       Freq      Total Category  logprob  loglift
76    question  38.000000  38.000000  Default  30.0000  30.0000
50       would  45.000000  45.000000  Default  29.0000  29.0000
73      number  27.000000  27.000000  Default  28.0000  28.0000
552  statement   8.000000   8.000000  Default  27.0000  27.0000
212      close  14.000000  14.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
69       fight   1.766416   6.029771   Topic8  -5.1834   1.3484
115       make   2.293579  14.850502   Topic8  -4.9222   0.7082
46       thing   2.525325  31.979591   Topic8  -4.8259   0.0374
661   industry   1.901543  16.499146   Topic8  -5.1096   0.4155
145   business   1.736391  19.162861   Topic8  -5.2005   0.1750

[495 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
246       3  0.826447  ability
305       6  0.820032    abuse
928       3  0.679584   access
611       2  0.328523  account
611       4  0.657046  account
...     ...       ...      ...
409       6  0.087844     year
409       7  0.146406     year
927       4  0.544472   youbut
964       4  0.510744     zone
964       7  0.255372     zone

[808 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 6, 3, 2, 5, 7, 4])

### Topic: First & Second debate

In [17]:
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_All, corpus_All, id2word_All)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.077678 -0.086712       1        1  17.876377
3     -0.042437 -0.104532       2        1  14.780819
7     -0.196126  0.041317       3        1  14.216921
2      0.045776 -0.054921       4        1  12.855190
0     -0.078587  0.012838       5        1  12.719456
6      0.136236 -0.020879       6        1  12.666403
1      0.176846 -0.030936       7        1   9.566026
5      0.035969  0.243825       8        1   5.318811, topic_info=          Term        Freq       Total Category  logprob  loglift
17    question   79.000000   79.000000  Default  30.0000  30.0000
62       would   89.000000   89.000000  Default  29.0000  29.0000
274  crosstalk   54.000000   54.000000  Default  28.0000  28.0000
57      people  199.000000  199.000000  Default  27.0000  27.0000
191     ballot   36.000000   36.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
308    support    5.170890   23.486553   Topic8  -4.4294   1.4205
976     policy    3.013020    7.651120   Topic8  -4.9695   2.0020
139    company    3.545729   12.526495   Topic8  -4.8067   1.6718
151       year    3.644686   68.024734   Topic8  -4.7792   0.0073
62       would    2.812518   89.105377   Topic8  -5.0384  -0.5218

[470 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
462       5  0.753689         ability
49        4  0.894377           abuse
1441      5  0.926196          accuse
750       2  0.972183         address
560       3  0.374374  administration
...     ...       ...             ...
151       5  0.088203            year
151       6  0.058802            year
151       7  0.117604            year
151       8  0.058802            year
1559      4  0.786067            zone

[683 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 8, 3, 1, 7, 2, 6])

## Evaluate LDA model

In [18]:

# Compute Coherence Score
coherence_model_lda_All = CoherenceModel(model=lda_model_All, texts=data_lemmatized_All, dictionary=id2word_All, coherence='c_v')
coherence_lda_All = coherence_model_lda_All.get_coherence()
print('Coherence Score for All debates   : ', coherence_lda_All)

coherence_model_lda_First = CoherenceModel(model=lda_model_First, texts=data_lemmatized_F, dictionary=id2word_F, coherence='c_v')
coherence_lda_F = coherence_model_lda_First.get_coherence()
print('Coherence Score for First debates : ', coherence_lda_F)

coherence_model_lda_Second = CoherenceModel(model=lda_model_Second, texts=data_lemmatized_S, dictionary=id2word_S, coherence='c_v')
coherence_lda_S = coherence_model_lda_Second.get_coherence()
print('Coherence Score for Second debates: ', coherence_lda_S)

Coherence Score for All debates   :  0.3867732979708409
Coherence Score for First debates :  0.4004375880497273
Coherence Score for Second debates:  0.3518270768582743


## Creat documents for caculate the topic similarity

In [19]:
df_T16=pd.read_csv('Data_files/Trump_2016.csv')

# Remove punctuation
df_T16['text_processed'] = df_T16['text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert lowercase
df_T16['text_processed'] = df_T16['text_processed'].map(lambda x: x.lower())


df_T20_1 = df_First[df_First.speaker=='Donald Trump']
df_T20_1.index = np.arange(0,len(df_T20_1))

df_T20_2 = df_Second[df_Second.speaker=='Donald Trump']
df_T20_2.index = np.arange(0,len(df_T20_2))

df_B20_1 = df_First[df_First.speaker=='Joe Biden']
df_B20_1.index = np.arange(0,len(df_B20_1))

df_B20_2 = df_Second[df_Second.speaker=='Joe Biden']
df_B20_2.index = np.arange(0,len(df_B20_2))

df_T16_1 = df_T16.iloc[:177]
df_T16_2 = df_T16.iloc[177:]

text_DT_2020_1 = " ".join(txt for txt in df_T20_1.text_processed)
text_DT_2020_2 = " ".join(txt for txt in df_T20_2.text_processed)
text_JB_2020_1 = " ".join(txt for txt in df_B20_1.text_processed)
text_JB_2020_2 = " ".join(txt for txt in df_B20_2.text_processed)
text_DT_2016_1 = " ".join(txt for txt in df_T16_1.text_processed)
text_DT_2016_2 = " ".join(txt for txt in df_T16_2.text_processed)

<>:4: DeprecationWarning: invalid escape sequence \.
<ipython-input-19-83c1c1a1f0d5>:4: DeprecationWarning: invalid escape sequence \.
  df_T16['text_processed'] = df_T16['text'].map(lambda x: re.sub('[,\.!?]', '', x))


## Topic similarity

#### Compute similarity of all Trump 2020 & Biden 2020 speach to 2020 Debate 1's topic

In [20]:
# Model: Debate 1, T vs B
test_doc = [text_DT_2020_1,text_DT_2020_2]
test_doc = [doc.split() for doc in test_doc]
test_corpus = [id2word_F.doc2bow(doc) for doc in test_doc]

# Cossim
doc1 = lda_model_First.get_document_topics(test_corpus[0], minimum_probability=0.1)
doc2 = lda_model_First.get_document_topics(test_corpus[1], minimum_probability=0.1)
print("Model: Debate 1, Trump 2020")
print(cossim(doc1, doc2))

test_doc2 = [text_JB_2020_1,text_JB_2020_2]
test_doc2 = [doc.split() for doc in test_doc2]
test_corpus2 = [id2word_F.doc2bow(doc) for doc in test_doc2]

doc3 = lda_model_First.get_document_topics(test_corpus2[0], minimum_probability=0.1)
doc4 = lda_model_First.get_document_topics(test_corpus2[1], minimum_probability=0.1)
print("Model: Debate 1, Biden 2020")
print(cossim(doc3, doc4))

Model: Debate 1, Trump 2020
0.9896990178560674
Model: Debate 1, Biden 2020
0.9214398702240059


#### Compute similarity of all Trump 2020 & Biden 2020 speach to 2020 Debate 2's topic

In [21]:
# Model: Debate 2, T vs B
test_doc = [text_DT_2020_1,text_DT_2020_2]
test_doc = [doc.split() for doc in test_doc]
test_corpus = [id2word_S.doc2bow(doc) for doc in test_doc]

# Cossim
doc1 = lda_model_Second.get_document_topics(test_corpus[0], minimum_probability=0.1)
doc2 = lda_model_Second.get_document_topics(test_corpus[1], minimum_probability=0.1)
print("Model: Debate 2, Trump 2020")
print(cossim(doc1, doc2))

test_doc2 = [text_JB_2020_1,text_JB_2020_2]
test_doc2 = [doc.split() for doc in test_doc2]
test_corpus2 = [id2word_S.doc2bow(doc) for doc in test_doc2]

doc3 = lda_model_Second.get_document_topics(test_corpus2[0], minimum_probability=0.1)
doc4 = lda_model_Second.get_document_topics(test_corpus2[1], minimum_probability=0.1)
print("Model: Debate 2, Biden 2020")
print(cossim(doc3, doc4))

Model: Debate 2, Trump 2020
0.7758101355600996
Model: Debate 2, Biden 2020
0.9978655682372872


#### Compute similarity of all Trump 2020 & Biden 2020 speach to 2020 All Debate's topic

In [22]:
# Model: Debate all, T vs B
test_doc = [text_DT_2020_1,text_DT_2020_2]
test_doc = [doc.split() for doc in test_doc]
test_corpus = [id2word_All.doc2bow(doc) for doc in test_doc]

# Cossim
doc1 = lda_model_All.get_document_topics(test_corpus[0], minimum_probability=0.1)
doc2 = lda_model_All.get_document_topics(test_corpus[1], minimum_probability=0.1)
print("Model: Debate all, Trump 2020")
print(cossim(doc1, doc2))

test_doc2 = [text_JB_2020_1,text_JB_2020_2]
test_doc2 = [doc.split() for doc in test_doc2]
test_corpus2 = [id2word_All.doc2bow(doc) for doc in test_doc2]

doc3 = lda_model_All.get_document_topics(test_corpus2[0], minimum_probability=0.1)
doc4 = lda_model_All.get_document_topics(test_corpus2[1], minimum_probability=0.1)
print("Model: Debate all, Biden 2020")
print(cossim(doc3, doc4))

Model: Debate all, Trump 2020
0.9843415868646007
Model: Debate all, Biden 2020
0.982587547258846


#### Compute similarity of all Trump 2020 & Trump 2016 speach to 2020 All Debate's topic

In [23]:
# Model: Debate all, T16 vs T20
test_doc = [text_DT_2020_1,text_DT_2020_2]
test_doc = [doc.split() for doc in test_doc]
test_corpus = [id2word_All.doc2bow(doc) for doc in test_doc]

# Cossim
doc1 = lda_model_All.get_document_topics(test_corpus[0], minimum_probability=0)
doc2 = lda_model_All.get_document_topics(test_corpus[1], minimum_probability=0)
print("Model: Debate all, Trump 2020")
print(cossim(doc1, doc2))

test_doc2 = [text_DT_2016_1,text_DT_2016_2]
test_doc2 = [doc.split() for doc in test_doc2]
test_corpus2 = [id2word_All.doc2bow(doc) for doc in test_doc2]

doc3 = lda_model_All.get_document_topics(test_corpus2[0], minimum_probability=0)
doc4 = lda_model_All.get_document_topics(test_corpus2[1], minimum_probability=0)
print("Model: Debate all, Trump 2016")
print(cossim(doc3, doc4))

Model: Debate all, Trump 2020
0.9847490657842577
Model: Debate all, Trump 2016
0.997191670406931
